In [10]:
#1 . Yahoo Finance

import yfinance as yf
import pandas as pd

def get_option_chain_yahoo(ticker):
    # Fetch the stock data from Yahoo Finance
    stock = yf.Ticker(ticker)
    
    # Fetch the expiration dates
    expiration_dates = stock.options
    
    # Dictionary to hold all the options data
    options_data = {'Calls': [], 'Puts': []}
    
    # Loop through each expiration date and fetch the options data
    for exp_date in expiration_dates:
        options = stock.option_chain(exp_date)
        
        # Add the expiration date to the data
        calls = options.calls.copy()
        calls['expirationDate'] = exp_date
        puts = options.puts.copy()
        puts['expirationDate'] = exp_date
        
        options_data['Calls'].append(calls)
        options_data['Puts'].append(puts)
    
    # Concatenate all the dataframes for calls and puts
    calls_df = pd.concat(options_data['Calls'])
    puts_df = pd.concat(options_data['Puts'])
    
    return calls_df, puts_df

# Example usage
ticker = 'AMZN'  # Apple Inc.
calls, puts = get_option_chain_yahoo(ticker)

# Display the first few rows of each dataframe
print("Calls:")
print(calls.head())
print("\nPuts:")
print(puts.head())



Calls:
        contractSymbol             lastTradeDate  strike  lastPrice    bid  \
0  AMZN240726C00100000 2024-07-19 19:26:40+00:00   100.0      83.31  82.70   
1  AMZN240726C00105000 2024-07-17 19:09:14+00:00   105.0      81.96  76.90   
2  AMZN240726C00110000 2024-07-18 18:44:18+00:00   110.0      74.18  72.00   
3  AMZN240726C00115000 2024-07-19 16:17:53+00:00   115.0      68.82  66.95   
4  AMZN240726C00120000 2024-07-19 19:46:42+00:00   120.0      62.60  62.00   

     ask    change  percentChange  volume  openInterest  impliedVolatility  \
0  83.90 -0.240006      -0.287260    22.0            59           2.126958   
1  79.55  0.000000       0.000000     1.0           104           1.828126   
2  74.50  0.000000       0.000000     1.0            35           1.738283   
3  69.55 -5.879997      -7.871482     1.0            12           1.601564   
4  64.50 -0.840000      -1.324086     3.0            29           1.468753   

   inTheMoney contractSize currency expirationDate  
0 

In [9]:
#https://dev.to/shahstavan/nse-option-chain-data-using-python-4fe5

# Libraries
import requests
import json
import math

# Python program to print
# colored text and background
def strRed(skk):         return "\033[91m {}\033[00m".format(skk)
def strGreen(skk):       return "\033[92m {}\033[00m".format(skk)
def strYellow(skk):      return "\033[93m {}\033[00m".format(skk)
def strLightPurple(skk): return "\033[94m {}\033[00m".format(skk)
def strPurple(skk):      return "\033[95m {}\033[00m".format(skk)
def strCyan(skk):        return "\033[96m {}\033[00m".format(skk)
def strLightGray(skk):   return "\033[97m {}\033[00m".format(skk)
def strBlack(skk):       return "\033[98m {}\033[00m".format(skk)
def strBold(skk):        return "\033[1m {}\033[0m".format(skk)

# Method to get nearest strikes
def round_nearest(x,num=50): return int(math.ceil(float(x)/num)*num)
def nearest_strike_bnf(x): return round_nearest(x,100)
def nearest_strike_nf(x): return round_nearest(x,50)

# Urls for fetching Data
url_oc      = "https://www.nseindia.com/option-chain"
url_bnf     = 'https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY'
url_nf      = 'https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'
url_indices = "https://www.nseindia.com/api/allIndices"

# Headers
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
            'accept-language': 'en,gu;q=0.9,hi;q=0.8',
            'accept-encoding': 'gzip, deflate, br'}

sess = requests.Session()
cookies = dict()

# Local methods
def set_cookie():
    request = sess.get(url_oc, headers=headers, timeout=5)
    cookies = dict(request.cookies)

def get_data(url):
    set_cookie()
    response = sess.get(url, headers=headers, timeout=5, cookies=cookies)
    if(response.status_code==401):
        set_cookie()
        response = sess.get(url_nf, headers=headers, timeout=5, cookies=cookies)
    if(response.status_code==200):
        return response.text
    return ""

def set_header():
    global bnf_ul
    global nf_ul
    global bnf_nearest
    global nf_nearest
    response_text = get_data(url_indices)
    data = json.loads(response_text)
    for index in data["data"]:
        if index["index"]=="NIFTY 50":
            nf_ul = index["last"]
            print("nifty")
        if index["index"]=="NIFTY BANK":
            bnf_ul = index["last"]
            print("banknifty")
    bnf_nearest=nearest_strike_bnf(bnf_ul)
    nf_nearest=nearest_strike_nf(nf_ul)

# Showing Header in structured format with Last Price and Nearest Strike

def print_header(index="",ul=0,nearest=0):
    print(strPurple( index.ljust(12," ") + " => ")+ strLightPurple(" Last Price: ") + strBold(str(ul)) + strLightPurple(" Nearest Strike: ") + strBold(str(nearest)))

def print_hr():
    print(strYellow("|".rjust(70,"-")))

# Fetching CE and PE data based on Nearest Expiry Date
def print_oi(num,step,nearest,url):
    strike = nearest - (step*num)
    start_strike = nearest - (step*num)
    response_text = get_data(url)
    data = json.loads(response_text)
    currExpiryDate = data["records"]["expiryDates"][0]
    for item in data['records']['data']:
        if item["expiryDate"] == currExpiryDate:
            if item["strikePrice"] == strike and item["strikePrice"] < start_strike+(step*num*2):
                #print(strCyan(str(item["strikePrice"])) + strGreen(" CE ") + "[ " + strBold(str(item["CE"]["openInterest"]).rjust(10," ")) + " ]" + strRed(" PE ")+"[ " + strBold(str(item["PE"]["openInterest"]).rjust(10," ")) + " ]")
                print(data["records"]["expiryDates"][0] + " " + str(item["strikePrice"]) + " CE " + "[ " + strBold(str(item["CE"]["openInterest"]).rjust(10," ")) + " ]" + " PE " + "[ " + strBold(str(item["PE"]["openInterest"]).rjust(10," ")) + " ]")
                strike = strike + step

# Finding highest Open Interest of People's in CE based on CE data         
def highest_oi_CE(num,step,nearest,url):
    strike = nearest - (step*num)
    start_strike = nearest - (step*num)
    response_text = get_data(url)
    data = json.loads(response_text)
    currExpiryDate = data["records"]["expiryDates"][0]
    max_oi = 0
    max_oi_strike = 0
    for item in data['records']['data']:
        if item["expiryDate"] == currExpiryDate:
            if item["strikePrice"] == strike and item["strikePrice"] < start_strike+(step*num*2):
                if item["CE"]["openInterest"] > max_oi:
                    max_oi = item["CE"]["openInterest"]
                    max_oi_strike = item["strikePrice"]
                strike = strike + step
    return max_oi_strike

# Finding highest Open Interest of People's in PE based on PE data 
def highest_oi_PE(num,step,nearest,url):
    strike = nearest - (step*num)
    start_strike = nearest - (step*num)
    response_text = get_data(url)
    data = json.loads(response_text)
    currExpiryDate = data["records"]["expiryDates"][0]
    max_oi = 0
    max_oi_strike = 0
    for item in data['records']['data']:
        if item["expiryDate"] == currExpiryDate:
            if item["strikePrice"] == strike and item["strikePrice"] < start_strike+(step*num*2):
                if item["PE"]["openInterest"] > max_oi:
                    max_oi = item["PE"]["openInterest"]
                    max_oi_strike = item["strikePrice"]
                strike = strike + step
    return max_oi_strike

set_header()
print('\033c')
print_hr()
print_header("Nifty",nf_ul,nf_nearest)
print_hr()
print_oi(10,50,nf_nearest,url_nf)
print_hr()
print_header("Bank Nifty",bnf_ul,bnf_nearest)
print_hr()
print_oi(10,100,bnf_nearest,url_bnf)
print_hr()

# Finding Highest OI in Call Option In Nifty
nf_highestoi_CE = highest_oi_CE(10,50,nf_nearest,url_nf)

# Finding Highet OI in Put Option In Nifty
nf_highestoi_PE = highest_oi_PE(10,50,nf_nearest,url_nf)

# Finding Highest OI in Call Option In Bank Nifty
bnf_highestoi_CE = highest_oi_CE(10,100,bnf_nearest,url_bnf)

# Finding Highest OI in Put Option In Bank Nifty
bnf_highestoi_PE = highest_oi_PE(10,100,bnf_nearest,url_bnf)


print(strCyan(str("Major Support in Nifty:")) + str(nf_highestoi_CE))
print(strCyan(str("Major Resistance in Nifty:")) + str(nf_highestoi_PE))
print(strPurple(str("Major Support in Bank Nifty:")) + str(bnf_highestoi_CE))
print(strPurple(str("Major Resistance in Bank Nifty:")) + str(bnf_highestoi_PE))



nifty
banknifty
c
 ---------------------------------------------------------------------|
 Nifty        =>   Last Price:  24530.9  Nearest Strike:  24550
 ---------------------------------------------------------------------|
25-Jul-2024 24050 CE [        1519 ] PE [       18178 ]
25-Jul-2024 24100 CE [       18524 ] PE [       73546 ]
25-Jul-2024 24150 CE [        2328 ] PE [       18661 ]
25-Jul-2024 24200 CE [       39705 ] PE [      115261 ]
25-Jul-2024 24250 CE [        3685 ] PE [       18600 ]
25-Jul-2024 24300 CE [       35797 ] PE [      109022 ]
25-Jul-2024 24350 CE [        7106 ] PE [       18271 ]
25-Jul-2024 24400 CE [       44826 ] PE [       78852 ]
25-Jul-2024 24450 CE [       15628 ] PE [       25427 ]
25-Jul-2024 24500 CE [      145041 ] PE [      210663 ]
25-Jul-2024 24550 CE [       24334 ] PE [       20854 ]
25-Jul-2024 24600 CE [      151223 ] PE [      124659 ]
25-Jul-2024 24650 CE [       44537 ] PE [       16012 ]
25-Jul-2024 24700 CE [      138539 ] PE [    